# Описание текущей версии программы Corrosion Tool

## Библиотеки

In [29]:
%pip install thermopack[2.1.0]


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
from dataclasses import dataclass, field
from typing import List
from thermopack.cubic import cubic
from matplotlib import pyplot as plt
import copy
import math

## Структуры

### Класс Tube_point

Класс Tube_point в Python представляет отрезок трубы. Он содержит различные свойства, описывающие характеристики материала, протекающего по трубе в этой конкретном отрезке.
Свойства класса Tube_point включают:
* **name**: строка, представляющая название материала.
* **phase_name**: список строк, представляющих названия фаз, присутствующих нутри отрезкае (например, газ, жидкость).
* **number_of_fluids**: целое число, указывающее количество присутствующих фаз.
* **temperature**: значение с плавающей точкой, представляющлокальнуюную температ,у в Кельынах.
* **pressure**: значение с плавающей точкой, представляющее локальное давл,иПаscal.
* **velocity**: значение с плавающей точкой, представляющее скорость смеси, протекающей по трм/скунду.
* **diameter**: значение с плавающей точкой, представляющее локальный диаметр,руетрыметрах.
* **length**: значение с плавающей точкой, представляющееотрезка трубый точке. Если это конечная точка, длина равна нулю.
* **section_type**: целое число, представляющее тип раздела для расчета потерь (в настоящее время не используется).
* **molar_composition**: список плавающих значений, представляющих молярный состав смеси.
* **molar_masses**: список значений с плавающей запятой, представляющих молярные массы компонентов смеси.
* **vapor_componnts**: список с плавающей точкой, представляющий распределение пара по компонентам (сумма равна 1).
* **liquid_components**: список значений с плавающей запятой, представляющих распределение жидкости по компонентам (сумма равна 1).
* **components_density**: список с плавающей точкой, представляющий распределение плотности по компонентам.
* **overall_density**: значение с плавающей точкой, представляющее общую п,тность смеси в кг/м^3.
* **overall_vapзначение с плавающей запятойion**: поплавок, представляющий распределение пара по смеси.
* **overall_liqзначение с плавающей запятойtion**: поплавок, представляющий распределение жидкости по смеси.
* **liquid_viscosities**: список плавающих значений, представляющих вязкость жидких компонентов по компонентам.
* **vapor_viscosities**: список значений с плавающей запятой, представляющих вязкость п по компонентамв над компонентами.
* **liquid_overall_viscosity**: значение с плавающей запятой, представляющее вязкость жидкой части.
* **vapor_overall_viscosity**: значение с плавающей запятой, представляющее вязкость паровой части.
* **overall_viscosity**: значение с плавающей точкой, представляющее вязкость смеси.
* **flow_mode**: строка, представляющая выбранный режим потока.
* **flow_mode_key**: значение с плавающей точкой, представляющее число, используемое для характеристики режима потока.
* **flow_mode_friction_factor**: значение с плавающей точкой, представляющее коэффициент трения в режиме потока.
* **reynolds_number**: число с плавающей точкой, представляющее число Рейнольдса для потока в данный момент.

In [46]:
@dataclass
class Tube_point:
    name: str = 'mixture'  # name: string, name of the material
    phase_name: List[str] = field(default_factory=lambda: ['gas'])  # phase_name: list of strings, name of phases
    number_of_fluids: int = 1  # number_of_fluids: integer, number of phases
    temperature: float = 300.0  # T: double, local temperature, Kelvin
    pressure: float = 101325.0  # p: double, local pressure, Pascal
    velocity: float = 0.5  # v: double, mixture velocity, m/s
    diameter: float = 0.1  # D: double, tube local diameter, m
    length: float = 10  # length: double, tube length, m. Zero if it's endpoint
    section_type = 1  # section type: integer, type of the section for losses calculation, not used yet - enum!
    molar_composition: List[float] = field(
        default_factory=lambda: [1.0])  # molar_composition: list of doubles, molar composition [probably constant]
    molar_masses: List[float] = field(
        default_factory=lambda: [1.0])  # molar_masses: list of doubles, molar masses [constant overall]
    vapor_components: List[float] = field(default_factory=lambda: [
        0.5])  # vapor_components: list of doubles, vapor distribution over components (sum is 1)
    liquid_components: List[float] = field(default_factory=lambda: [
        0.5])  # liquid_components: list of doubles, liquid distribution over components (sum is 1)
    components_density: List[float] = field(
        default_factory=lambda: [1.0])  # components_density: list of doubles, density distribution over components
    overall_density: float = 1.0  # overall_density: double, overall density, kg/m^3
    overall_vapor_fraction: float = 0.5  # overall_vapor_fraction: double, vapor distribution over mixture
    overall_liquid_fraction: float = 0.5  # overall_liquid_fraction: double, liquid distribution over mixture
    liquid_viscosities: List[float] = field(default_factory=lambda: [
        1.0e-3])  # liquid_viscosities: list of doubles, viscosity of liquid parts over components
    vapor_viscosities: List[float] = field(default_factory=lambda: [
        1.0e-3])  # vapor_viscosities:  list of doubles, viscosity of vapor parts over components
    liquid_overall_viscosity: float = 1.0e-3  # liquid_overall_viscosity: double, viscosity of liquid part
    vapor_overall_viscosity: float = 1.0e-3  # vapor_overall_viscosity: double, viscosity of vapor part
    overall_viscosity: float = 1.0e-3  # overall_viscosity: double, viscosity of mixture
    flow_mode: str = "bubble"  # flow_mode: string, name of selected flow flow_mode
    flow_mode_key: float = 1.0  # flow_mode_key: double, currently XTT, later - other number to characterize flow_mode
    flow_mode_friction_factor: float = 1.0  # flow_mode_friction_factor: double, currently from XTT
    reynolds_number: float = 10000.0  # reynolds_number: double, Reynolds number for ...

    def update_point_state(self):
        rk_fluid = cubic('N2,ETOH', 'SRK')  # obsolete
        x, y, vap_frac, liq_frac, phase_key = rk_fluid.two_phase_tpflash(self.temperature, self.pressure,
                                                                         self.molar_composition)
        self.vapor_components = x
        self.liquid_components = y
        self.overall_vapor_fraction = vap_frac
        self.overall_liquid_fraction = liq_frac

        temp, = rk_fluid.specific_volume(self.temperature, self.pressure, self.molar_composition, 1)
        density_1 = self.molar_masses[0] / temp

        temp, = rk_fluid.specific_volume(self.temperature, self.pressure, self.molar_composition, 2)
        density_2 = self.molar_masses[1] / temp

        self.components_density = [density_1, density_2]
        self.overall_density = self.calculate_overall_density()
        ethanol_viscosity = ethanol_viscosity_from_temperature(self.temperature)
        n2_viscosity = n2_viscosity_from_temperature(self.temperature)
        self.liquid_viscosities = [ethanol_viscosity, n2_viscosity]
        self.vapor_viscosities = [ethanol_viscosity, n2_viscosity]

    def calculate_Re(self):
        return self.velocity * self.diameter * self.overall_density / self.overall_viscosity

    def calculate_xtt(self):
        liquid_density = self.components_density[0]
        gas_density = self.components_density[1]
        liquid_viscosity = self.liquid_viscosities[0]  # ? liquid_overall_viscosity?
        gas_viscosity = self.liquid_viscosities[1]  # ?
        velocity = self.velocity
        diameter = self.diameter
        return ((1.096 / liquid_density) ** 0.5) * ((liquid_density / gas_density) ** 0.25) * (
                (gas_viscosity / liquid_viscosity) ** 0.1) * ((velocity / diameter) ** 0.5)

    def calculate_overall_density(self):  # необходимо дописать учёт агрегатного состояния
        return sum(self.molar_composition[i] * self.components_density[i] for i in range(self.number_of_fluids))

    def calculate_lambda(self):
        if self.reynolds_number < 2300:
            return 64 / self.reynolds_number
        else:
            return 0.316 / (self.reynolds_number ** 0.25)

    def calculate_pressure_loss(self):
        xi = self.calculate_lambda() * self.length / self.diameter
        return (xi * self.velocity ** 2) * 0.5 * self.overall_density
point = Tube_point()
point.temperature = 320.0
point.pressure = 354637.5
point.molar_composition = [0.5,0.5]
point.molar_masses = [0.03,0.28]
point.velocity = 5.0
point.diameter = 0.08
point.length = 100

#### Метод calculate_overall_density

Вычисляет плотность среды. Реализация одна. На вход подаётся структура данных, из которой берутся мольные доли фаз и плотности фаз (без учета агрегатного состояния, необходимо дописать учёт агрегатного состояния). Далее вычисление происходит по формуле:
$$
\rho_{overall}=\sum\rho_ix_i
$$

In [32]:
print("Overall density : " + str(point.calculate_overall_density()))

Overall density : 0.5


#### Метод update_point_state

Выполняет расчёты текущих свойств (вязкости – внешними свойствами, см. выше), плотность и агрегатное состояние фаз – через thermopack;). Вызывается при изменении параметров температуры и давления для корректировки остальных характеристик трубы.

## Функции

### Функция calculate_xtt¶

Рассчитывает параметр, по которому можно получить режим течения. Применена простейшая корреляция, в дальнейшем потребует корректировки.
Есть 2 реализации, одна- с параметрами, вторая – со структурой, эти 2 реализации не отличаются ничем, кроме способа хранения данных. На вход подаётся плотности и вязкости жидкой и газообразной фазы, а также скорость среды и диаметр трубы. На выходе – параметр xtt (параметр Локхарта-Мартинелли), который рассчитывается как

$$ 
Xtt=\sqrt{\frac{1.096}{\rho_{Liquid}}}\cdot\left[\frac{\rho_{Liquid}}{\rho_{Gas}}\right]^{0.25}\cdot\left[\frac{\mu_{Gas}}{\mu_{Liquid}}\right]^{0.1}\cdot\sqrt{\frac{V_{Liquid}}{D}}
$$

In [33]:
def calculate_xtt(liquid_density, gas_density, liquid_viscosity, gas_viscosity, velocity, diameter):
    return ((1.096 / liquid_density) ** 0.5) * ((liquid_density / gas_density) ** 0.25) * (
            (gas_viscosity / liquid_viscosity) ** 0.1) * ((velocity / diameter) ** 0.5)
    
print("XTT: "+str(calculate_xtt(397.3 ,3.9 ,0.001, 0.0001, 5.0, 0.08)))

XTT: 1.047850904778871


### Функция calculate_viscosity

Вычисляет общую вязкость среды по вязкости жидкой и газообразной фаз и параметру friction factor. Реализация одна. На вход подаются 2 значения вязкости (жидкости и газа) и параметр friction factor (ff). Далее вычисление происходит по формуле:
$$
\mu_{Total}=ff\cdot\mu_{Liquid}+\left(1-ff\right)\cdot\mu_{Gas}$$


In [34]:
def calculate_viscosity(liquid_viscosity, gas_viscosity, friction_factor):
    return friction_factor * liquid_viscosity + (1 - friction_factor) * gas_viscosity

print("Viscosity: "+str(calculate_viscosity(0.001, 0.0001, 1)))

Viscosity: 0.001


### Функция calculate_Re

Вычисляет число Рейнольдса по общей плотности и общей вязкости среды. Есть 2 реализации, одна- с параметрами, вторая – со структурой, эти 2 реализации не отличаются ничем, кроме способа хранения данных. На вход подаются общая вязкость среды, общая плотность среды, диаметр трубы и скорость среды. Вычисление происходит по формуле:
$$
Re=\frac{v\cdot D\cdot\rho}{\mu}
$$

In [35]:
def calculate_Re(velocity, diameter, overall_density, overall_viscosity):
    return velocity * diameter * overall_density / overall_viscosity

print("Re : "+str(calculate_Re(5.0, 0.08, 198.6, 7.1e-07)))

Re : 111887323.94366197


### Функция calculate_lambda

Вычисляет коэффициент трения для потерь по длине. Реализация одна. Реализовано без оглядки на режимы многофазного течения, но с простейшим учетом ламинарного и турбулентного режимов течения в трубе. На вход подаётся значение числа Рейнольдса. Далее вычисление происходит по формуле:
$$
\lambda=\begin{cases}\frac{64}{Re}  если  Re<2300\\ \frac{0.316}{Re^{0.25}}  если  Re≥2300\end{cases}
$$

In [36]:
def calculate_lambda(Re):
    if Re < 2300:
        return 64 / Re
    else:
        return 0.316 / (Re ** 0.25)

print("Lambda : "+str(calculate_lambda(111887323.9)))

Lambda : 0.0030724997835867044


### Функция return_pressure_loss

Вычисляет потери по длине. Реализация одна. На вход подается значение коэффициента трения, скорость, плотность, диаметр трубы и длина участка трубы. Далее вычисление происходит по формуле:
$$
\begin{cases}ξ=\frac{λ⋅L}{D}\\ΔP=ξ⋅\frac{ρV^2}{ 2}\end{cases}
$$

In [37]:
def return_pressure_loss(velocity, diameter, length, lam, density):
    xi = lam * length / diameter
    return (xi * velocity ** 2) * 0.5 * density

print("Pressure loss : "+str(return_pressure_loss(5.0, 0.08, 100, 0.003, 198.6)))

Pressure loss : 9309.375


### Функция return_mode

По значению xtt выводит строку с режимом течения. Реализация одна.
На вход подаётся параметр xtt.

In [38]:
def return_mode(xtt):
    if xtt < 10: return 'bubble'
    if 10 <= xtt < 100:
        return 'plug'
    if 100 <= xtt < 1000:
        return 'slug'
    if 1000 <= xtt < 10000:
        return 'annular'
    if 10000 <= xtt:
        return 'mist'
    return 'undefined'
print("Mode : "+str(return_mode(7.6)))

Mode : bubble


### Функция return_friction_factor

По значению xtt выводит friction factor для расчёта вязкости. **Значения для разных режимов взяты с потолка.** На вход подаётся параметр xtt.

In [39]:
def return_friction_factor(xtt):
    """
    Outputs the friction factor to calculate the viscosity.
    :param xtt:
    :return:
    """
    if xtt < 10:
        return 1
    if 10 <= xtt < 100:
        return 0.9
    if 100 <= xtt < 1000:
        return 0.8
    if 1000 <= xtt < 10000:
        return 0.7
    if 10000 <= xtt:
        return 0.6
    return 0
print("Friction factor : "+str(return_friction_factor(7.6)))

Friction factor : 1


### Функция ethanol_viscosity_from_temperature

Вычисление вязкости жидкого этанола в зависимости от температуры по экспоненциальной корреляции. Функция является временной и должна быть заменена на более общую корреляцию. На вход подаётся температура, на выходе – вязкость в Па*с. Постоянные для этанола зашиты внутрь самой функции:
$$
\mu_{Ethanol}=A\exp{\left(\frac{B}{T}+CT+DT^2\right)}
$$
$$
\begin{cases}A = 0.00201 mPa⋅s\\B = 1614\cdot K\\C = 0.00618\cdot K-1\\D = -1.132⋅10^{-5}\cdot K^{-2}\end{cases}
$$

In [40]:
def ethanol_viscosity_from_temperature(T):
    A = 0.00201 * 1e-6
    B = 1614
    C = 0.00618
    D = 1.132 * (-1e-5)
    return A * math.exp(B / T + C * T + D * T * T)
    
print("Ethanol viscosity : "+str(ethanol_viscosity_from_temperature(307.5)))

Ethanol viscosity : 8.77334610818247e-07


### Функция n2_viscosity_from_temperature

вычисление вязкости газообразного азота в зависимости от температуры по формуле Сазерленда.  Функция является временной и должна быть заменена на более общую корреляцию. На вход подаётся температура, на выходе – вязкость в Па*с. Постоянные для азота зашиты внутрь самой функции:
$$
\mu_{Nitrogen}=\mu_0\left(\frac{T}{T_0}\right)^\frac{3}{2}\cdot\frac{T_0+S}{T+S}
$$
$$
\begin{cases}μ0=1.7⋅10-5 Pa⋅s\\ T0=273 K\\S=104.7 K \end{cases}
$$

In [41]:
def n2_viscosity_from_temperature(T):
    VISCOSITY_INIT = 1.7e-5
    T_INIT = 273
    S = 104.7
    return (VISCOSITY_INIT * (T / T_INIT) ** 1.5) * (T_INIT + S) / (T + S)

print("Nitrogen viscosity : "+str(n2_viscosity_from_temperature(307.5)))

Nitrogen viscosity : 1.8621369952874267e-05


### Функция start_point и fill_variables_from_json

Инициализация и заполнение структуры данных начальными параметрами. Аналог функции pvt_block, но для первой итерации. Заполняется из файла.

In [42]:
def start_point(point: Tube_point, path: str):

    fill_variables_from_json(path, point)
    point.update_point_state()
    return point


def fill_variables_from_json(json_file, point):
    with open(json_file, 'r') as file:
        data = json.load(file)

    point.temperature = data.get('temperature', point.temperature)
    point.pressure = data.get('pressure', point.pressure)
    point.molar_composition = data.get('molar_composition', point.molar_composition)
    point.molar_masses = data.get('molar_masses', point.molar_masses)
    point.velocity = data.get('velocity', point.velocity)
    point.diameter = data.get('diameter', point.diameter)
    point.length = data.get('length', point.length)

### Функция define_tube_params

функция, задающая в структуре данных параметры трубы и пересчитывающая скорость через сохранение расхода. Принимает объект структуры данных, диаметр трубы и длину участка, а так же плотность на предыдущем шаге. Расход на предыдущем шаге вычисляется как:
$$
{Q^\prime}_{i-1}=4\cdot\frac{Q_{i-1}}{\pi}=v_{i-1}\cdot D_{i-1}^2\cdot\rho_{i-1}
$$
Тогда скорость из расхода (при учете его сохранения, т.е. выполнения уравнения неразрывности:)
$$
Q_{i-1}^\prime=Q_i^\prime\rightarrow v_i=\frac{Q_i^\prime}{D_i^2\cdot\rho_i}\ 
$$

In [43]:
def define_tube_params(point: Tube_point, diameter, length, density_old):
    q = point.velocity * point.diameter * point.diameter * density_old
    new_velocity = q / (diameter * diameter * point.overall_density)  # mass balance, pi/4 is skipped
    # due to presence in both parts of equation
    point.diameter = diameter
    point.length = length
    point.velocity = new_velocity
    return point

print("Ethanol viscosity : "+str(define_tube_params(point,0.08, 100, 13.5)))

Ethanol viscosity : Tube_point(name='mixture', phase_name=['gas'], number_of_fluids=1, temperature=320.0, pressure=354637.5, velocity=67.5, diameter=0.08, length=100, molar_composition=[0.5, 0.5], molar_masses=[0.03, 0.28], vapor_components=[0.5], liquid_components=[0.5], components_density=[1.0], overall_density=1.0, overall_vapor_fraction=0.5, overall_liquid_fraction=0.5, liquid_viscosities=[0.001], vapor_viscosities=[0.001], liquid_overall_viscosity=0.001, vapor_overall_viscosity=0.001, overall_viscosity=0.001, flow_mode='bubble', flow_mode_key=1.0, flow_mode_friction_factor=1.0, reynolds_number=10000.0)


### Функция pvt_block

Основная рабочая функция. Меняет основные показатели и обновляет состояние отрезка трубы.

In [44]:
def pvt_block(point: Tube_point, new_pressure, new_temperature):
    point.temperature = new_temperature
    point.pressure = new_pressure
    point.update_point_state()
    return point
pvt_block(point, 310.5, 400000)

Tube_point(name='mixture', phase_name=['gas'], number_of_fluids=1, temperature=400000, pressure=310.5, velocity=67.5, diameter=0.08, length=100, molar_composition=[0.5, 0.5], molar_masses=[0.03, 0.28], vapor_components=array([0.5, 0.5]), liquid_components=array([0., 0.]), components_density=[2.8008425108725974e-06, 2.6141196768144248e-05], overall_density=1.4004212554362987e-06, overall_vapor_fraction=-1.0, overall_liquid_fraction=-1.0, liquid_viscosities=[0.0, 0.0009000533850548925], vapor_viscosities=[0.0, 0.0009000533850548925], liquid_overall_viscosity=0.001, vapor_overall_viscosity=0.001, overall_viscosity=0.001, flow_mode='bubble', flow_mode_key=1.0, flow_mode_friction_factor=1.0, reynolds_number=10000.0)

## Исполнение

Функция main выглядит так

In [51]:
point1 = Tube_point()
point1.temperature = 320.0
point1.pressure = 354637.5
point1.molar_composition = [0.5,0.5]
point1.molar_masses = [0.03,0.28]
point1.velocity = 5.0
point1.diameter = 0.08
point1.length = 100
point1.update_point_state()

def main():
    #point = Tube_point()
    # start_point(point, path)
    tube_diameters = [0.08, 0.08, 0.08, 0.09, 0.09, 0.09, 0.09, 0.08, 0.08, 0.1]
    tube_lengths = [1000, 200, 450, 1000, 200, 300, 200, 300, 1000, 0]
    tube_points = [point1]

    re_nums = list()
    list_p1 = list()
    list_t1 = list()

    for i in range(1, len(tube_diameters)):
        xtt = tube_points[i - 1].calculate_xtt()
        # print ('i', i, ' xtt ', xtt)
        flow_mode = return_mode(xtt)
        print('Found ', flow_mode, ' flow_mode at', i, '! xtt= ', xtt)
        friction_factor = return_friction_factor(xtt)
        tube_points[i - 1].overall_viscosity = calculate_viscosity(tube_points[i - 1].liquid_viscosities[0],
                                                                   tube_points[i - 1].liquid_viscosities[1],
                                                                   friction_factor)
        tube_points[i - 1].reynolds_number = tube_points[i - 1].calculate_Re()
        re_nums.append(tube_points[i - 1].reynolds_number)
        print('Reynolds number for ', i, ' is ', tube_points[i - 1].reynolds_number, 'lambda is ',
              calculate_lambda(tube_points[i - 1].reynolds_number))
        diff = point.calculate_pressure_loss()
        P1 = tube_points[i - 1].pressure - diff
        T1 = tube_points[i - 1].temperature - i * 0.3
        list_p1.append(P1)
        list_t1.append(T1)
        print('P1: ', P1, ', T1: ', T1)
        next_point = copy.deepcopy(tube_points[i - 1])

        density = next_point.overall_density
        next_point = pvt_block(next_point, P1, T1)
        next_point = define_tube_params(next_point, tube_diameters[i], tube_lengths[i], density)

        tube_points.append(next_point)
main()

Found  bubble  flow_mode at 1 ! xtt=  1.0351388096966774
Reynolds number for  1  is  112461849.56258014 lambda is  0.0030685681796535523
P1:  354143.75 , T1:  319.7
Found  bubble  flow_mode at 2 ! xtt=  1.0326423664700026
Reynolds number for  2  is  111895195.09999605 lambda is  0.0030724457488753632
P1:  353650.0 , T1:  319.09999999999997
Found  bubble  flow_mode at 3 ! xtt=  1.027357042336603
Reynolds number for  3  is  110768157.08520386 lambda is  0.0030802314447932015
P1:  353156.25 , T1:  318.2
Found  bubble  flow_mode at 4 ! xtt=  0.8543502212084051
Reynolds number for  4  is  96971727.4438501 lambda is  0.003184386686917118
P1:  352662.5 , T1:  317.0
Found  bubble  flow_mode at 5 ! xtt=  0.8457077754402509
Reynolds number for  5  is  95012251.2330759 lambda is  0.003200679442942756
P1:  352168.75 , T1:  315.5
Found  bubble  flow_mode at 6 ! xtt=  0.8352660231728324
Reynolds number for  6  is  92603715.17640795 lambda is  0.003221291154069259
P1:  351675.0 , T1:  313.7
Found  bu